In [11]:
import torch
import torch.nn as nn
import os
import random

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from third_party.midi_processor.processor import decode_midi, encode_midi

from utilities.argument_funcs import parse_generate_args, print_generate_args
from model.music_transformer import MusicTransformer
from dataset.e_piano import create_epiano_datasets, compute_epiano_accuracy, process_midi
from torch.utils.data import DataLoader
from torch.optim import Adam

from utilities.constants import *
from utilities.device import get_device, use_cuda

import argparse

In [30]:
parser = argparse.ArgumentParser()

parser.add_argument("-classic_input_dir", type=str, default="./dataset/e_piano", help="Folder of preprocessed and pickled midi files")
parser.add_argument("-pop_input_dir", type=str, default="./dataset/pop_pickle", help="Folder of preprocessed and pickled midi files")
parser.add_argument("-output_dir", type=str, default="./saved_models", help="Folder to save model weights. Saves one every epoch")
parser.add_argument("-weight_modulus", type=int, default=10, help="How often to save epoch weights (ex: value of 10 means save every 10 epochs)")
parser.add_argument("-print_modulus", type=int, default=50, help="How often to print train results for a batch (batch loss, learn rate, etc.)")

parser.add_argument("-n_workers", type=int, default=4, help="Number of threads for the dataloader")
parser.add_argument("--force_cpu", action="store_true", help="Forces model to run on a cpu even when gpu is available")
parser.add_argument("--no_tensorboard", action="store_true", help="Turns off tensorboard result reporting")

parser.add_argument("--gan", action="store_true", help="use generative adversarial training")
parser.add_argument("--creative", action="store_true", help="creative learning")

parser.add_argument("-continue_weights", type=str, default=None, help="Model weights to continue training based on")
parser.add_argument("-continue_epoch", type=int, default=None, help="Epoch the continue_weights model was at")

parser.add_argument("-lr", type=float, default=None, help="Constant learn rate. Leave as None for a custom scheduler.")
parser.add_argument("-ce_smoothing", type=float, default=0.1, help="Smoothing parameter for smoothed cross entropy loss (defaults to no smoothing)")
parser.add_argument("-batch_size", type=int, default=8, help="Batch size to use")
parser.add_argument("-epochs", type=int, default=100, help="Number of epochs to use")

parser.add_argument("--rpr", action="store_true", help="Use a modified Transformer for Relative Position Representations")
parser.add_argument("-max_sequence", type=int, default=1536, help="Maximum midi sequence to consider")
parser.add_argument("-n_layers", type=int, default=6, help="Number of decoder layers to use")
parser.add_argument("-num_heads", type=int, default=8, help="Number of heads to use for multi-head attention")
parser.add_argument("-d_model", type=int, default=512, help="Dimension of the model (output dim of embedding layers, etc.)")

parser.add_argument("-dim_feedforward", type=int, default=1024, help="Dimension of the feedforward layer")
parser.add_argument("-num_prime", type=int, default=256, help="Amount of messages to prime the generator with")

parser.add_argument("-target_seq_length", type=int, default=1024, help="Target length you'd like the midi to be")
# parser.add_argument("-num_prime", type=int, default=256, help="Amount of messages to prime the generator with")
# parser.add_argument("-model_weights", type=str, default="./saved_models/model.pickle", help="Pickled model weights file saved with torch.save and model.state_dict()")
parser.add_argument("-beam", type=int, default=0, help="Beam search k. 0 for random probability sample and 1 for greedy")

parser.add_argument("-dropout", type=float, default=0.1, help="Dropout rate")

args = parser.parse_args(args=[])

In [31]:
args.rpr = True

In [32]:

# argument parsing\

if(args.force_cpu):
    use_cuda(False)
    print("WARNING: Forced CPU usage, expect model to perform slower")
    print("")

# 저장할 directory 만들기
os.makedirs(args.output_dir, exist_ok=True)
os.makedirs(args.output_dir+'/classic/', exist_ok=True)
os.makedirs(args.output_dir+'/pop/', exist_ok=True)

# Grabbing dataset if needed
# pickle file path - EY
classic_path = './dataset/e_piano/'
pop_path = './dataset/pop_trainvalid/'

# train, val, test
classic_train, classic_eval, classic_test = create_epiano_datasets(classic_path, args.num_prime, random_seq=False)
pop_train, pop_eval, pop_test = create_epiano_datasets(pop_path, args.num_prime, random_seq=False)

classic_dataset = [classic_train, classic_eval, classic_test]
pop_dataset = [pop_train, pop_eval, pop_test]
dataset_folder = ['train/', 'val/', 'test/']

# Can be None, an integer index to dataset, or a file path
# if(args.primer_file is None):
#     f = str(random.randrange(len(dataset)))
# else:
#     f = args.primer_file

# if(f.isdigit()):
#     idx = int(f)
#     primer, _, _  = dataset[idx]
#     primer = primer.to(get_device())
#
#     print("Using primer index:", idx, "(", dataset.data_files[idx], ")")
#
# else:
#     raw_mid = encode_midi(f)
#     if(len(raw_mid) == 0):
#         print("Error: No midi messages in primer file:", f)
#         return
#
#     primer, _  = process_midi(raw_mid, args.num_prime, random_seq=False)
#     primer = torch.tensor(primer, dtype=TORCH_LABEL_TYPE, device=get_device())
#
#     print("Using primer file:", f)



In [38]:
# pop generation
for dataset, folder in zip(pop_dataset, dataset_folder):

    pop_index_list = list(range(len(dataset)))
    folder_name_length = len(folder)

    for pop_index in pop_index_list:
        primer, _, _ = dataset[pop_index]
        primer = primer.to(get_device())
        print("Using primer index:", pop_index, "(", dataset.data_files[pop_index], ")")

        # # Saving primer first
        # f_path = os.path.join(args.output_dir, f"primer_{pop_dataset.data_files[pop_index][len(classic_path)+5:]}.mid")
        # decode_midi(primer[:args.num_prime].cpu().numpy(), file_path=f_path)

        print("RAND DIST")
#         rand_seq = model.generate(primer[:args.num_prime], args.target_seq_length, beam=0)

        f_path = os.path.join(
            args.output_dir+'/pop/', f"rand_{dataset.data_files[pop_index][len(pop_path)+folder_name_length:]}.mid")
        decode_midi(primer.cpu().numpy(), file_path=f_path)
        print("Done!")
        break

Using primer index: 0 ( ./dataset/pop_trainvalid/train/883.pickle )
RAND DIST
Done!
Using primer index: 0 ( ./dataset/pop_trainvalid/val/109.pickle )
RAND DIST
Done!
Using primer index: 0 ( ./dataset/pop_trainvalid/test/080.pickle )
RAND DIST
Done!


In [36]:
len(primer)

256

In [39]:
f_path

'./saved_models/pop/rand_080.pickle.mid'